# HW4 - Convert WikiHow to PDDL

In this homework we will translate a wikHow article into the Planning Domain Definition Language (PDDL).    The goals of this homework are to:
1. Deepen our understanding of classical planning
2. Understand the process of manually translating from natural language instructions into PDDL.
3. Think about how we might teach GPT-3 to perform the translation automatically (don't worry – you don't have to implement the automatic translation in this homework).
4. Capture some data that might be useful for fine-tuning GPT-3 to convert to PDDL (for possible use in some students' term projects).  

This colab notebook will give you an example of how to convert one step from a wikiHow article into PDDL.  You'll then do this on your own and submit a domain.pddl and several problem.pddl files.

In [ ]:
# Install the PDDL parser
!git clone https://github.com/pucrs-automated-planning/pddl-parser &> /dev/null
# !cd pddl-parser

# Make the directory available to Python so you can import packages from it.
import sys
sys.path.append("pddl-parser")

In [ ]:
# # Mount your Google Drive so that you can save your PDDL files.
# from google.colab import drive
# drive.mount('/content/gdrive')
# path = '/content/gdrive/Shareddrives/workspace/22sp/cis700/hw4'
path = './'

These are some helper functions for loading and saving files, and for displaying long texts.

In [ ]:
import os
import json
import textwrap


## Create a file from the contents of a string. 
def write_to_file(string, path, filename):
  """This is a helper function to create a file from a string."""
  if not os.path.exists(path) and path != "":
    os.makedirs(path)
  with open(os.path.join(path, filename), "w") as f:
      f.write(string)


# Load a JSON file given a filename
# If the file doesn't exist, then return an empty dictionary instead
def load_json(filename):
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return {}

# Save a JSON file given a full path filename and a dictionary
# If the path doesn't exist, create the directories before saving the file.
def save_json(filename, data):
    if not os.path.exists(os.path.dirname(filename)):
        os.makedirs(os.path.dirname(filename))
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)


def wrap_text(text):
  wrapped_text = ""
  lines = text.strip().split("\n")

  for line in lines:
    if len(line) > 90:
        w = textwrap.TextWrapper(width=90, break_long_words=False)
        line = '\n'.join(w.wrap(line))
    wrapped_text += line + "\n"
  return wrapped_text


# Step 1: Pick an Interesting wikiHow Article

We'll create our PDDL files from a wikiHow article.  The goal for this is to start from something that describes proceedures and actions and is written in natural language, and then to manually translate it into the description language used for automated planning.

Our selected wikiHow article and specific part of interest
* [How to Survive in the Woods - Part 4: Starting a Fire](https://www.wikihow.com/Survive-in-the-Woods#:~:text=4-,Starting%20a%20Fire,-Download%20Article)

# Step 2: Manually Create a PDDL


## Domain and Problems

You should create PDDL files for:
1. The domain.  You should have a single `domain.pddl` file which defines the domain, including the types, predicates and action schemata that are relevant to your wikiHow article.
2. One or more problems.  You should create one more problem file that defines a problem, an initial state, and a goal, that can be reached by applying the action schema that you defined.  In some cases, it might make sense to have one problem file for each step in a wikiHow article. 

The name of your domain should be the aricle title.  I have started defining a domain for `survive_in_the_woods`.
```
(define (domain survive_in_the_woods)
   (:requirements :strips :typing)
   ...
```   
I'll create a problem file for Step 1 in the article.  I'll name the problem `collect_water` and I'll define the goal as `(:goal (and (inventory npc water))` where the player (or an AI-controled non-playable character, abbreviated as NPC) needs to add water to their inventory. 

Here's what the start and end of the problem PDDL file would look like:

```
(define (problem collect_water)
   (:domain survive_in_the_woods)
...
   (:goal (and (inventory npc water)))
```


## Types

You should specify what the `types` are in your domain.  They should be the things that are relevant to solving the problem that you're tackling.  It's sometimes also useful to create subtypes. Which you can do like this:
```
   (:types        
       water - item
       player direction location 
   )
``` 
By default all types are subtypes of `object`. The line
```
water - item
```
allows us to define `water` as a subtype of `item` (`item` is also introduced as a type in this same line).   Having `water` as a subtype allows us to restrict some action schemas to only operate on that type.


## Action Schema

Here's an example of an action schema for getting items:
```
   (:action get
      :parameters (?item - item ?p - player ?l1 - location) 
      :precondition (and (at ?p ?l1) (at ?item ?l1))
      :effect (and (inventory ?p ?item) (not (at ?item ?l1)))
   )
```
It has the effect or removing the item from the current location and putting it into the player's inventory.

We might add a seperate action for getting water, since when our player gets some water from a lake, the water shouldn't be removed from that location.  Here's one way to write it:

```
   (:action get_water
      :parameters (?p - player ?loc - location ?water - water) 
      :precondition (and (at ?p ?loc) (has_water_source ?loc))
      :effect (and (inventory ?p ?water) (not (treated ?water)))
   )
```
(We might also consider adding some additional preconditions, like that the player has a container to store their water in). 

## Predicates

In your PDDL domain file, please define your predicates like this:
```
   (:predicates
      (has_water_source ?loc - location) ; this location has a source of fresh water.
      (treated ?water - water) ; True if the water has been decontaimated by boiling it
      ...
   )
```
You should give the type of each predicate's arguments, and a comment about what the predicate means (everything after the `;` is a comment). 

## Domain 

In [ ]:
domain = """
(define (domain survive_in_the_woods)
   (:requirements :strips :typing)
   (:types
       grass branch log tinder kindling fuel teepee wood water pot - item
       player direction location
   )

   (:predicates
      (has_large_dry_branch ?loc - location) ; this location has large and dry branches that can be broken off as kindling material.
      (has_dry_oak ?loc - location)          ; this location has dry oak trees that can be logged for fuel.
      (has_dry_even_area ?loc - location)    ; this location has a dry, even area without things that could catch on fire.
      (has_pond ?loc - location)             ; this location has a pond with water
      (dry ?item)                            ; True if the item is dry
      (ignited ?item)                        ; True if the item is ignited
      (boiled ?item)                         ; True if the item (namely water) has been boiled
      (at ?obj - object ?loc - location)     ; an object is at a location 
      (inventory ?player ?item)              ; an item is in the player's inventory
      (connected ?loc1 - location ?dir - direction ?loc2 - location) ; location 1 is connected to location 2 in the direction
      (blocked ?loc1 - location ?dir - direction ?loc2 - location)   ; the connection between location 1 and 2 in currently blocked
   )

   (:action go ; navigate to an adjacent location 
      :parameters (?dir - direction ?p - player ?l1 - location ?l2 - location) 
      :precondition (and (at ?p ?l1) (connected ?l1 ?dir ?l2) (not (blocked ?l1 ?dir ?l2)))
      :effect (and (at ?p ?l2) (not (at ?p ?l1)))
   )

   (:action get ; pick up an item and put it in the inventory
      :parameters (?item - item ?p - player ?l1 - location) 
      :precondition (and (at ?p ?l1) (at ?item ?l1))
      :effect (and (inventory ?p ?item) (not (at ?item ?l1)))
   )

   (:action get_grass ; gather grass that is not dry at the first time
      :parameters (?p - player ?loc - location ?grass - grass)
      :precondition (and (at ?p ?loc) (at ?grass ?loc))
      :effect (and (inventory ?p ?grass) (not (dry ?grass)))
   )

   (:action dry_grass ; dry grass so that it can used as tinder
      :parameters (?p - player ?grass - grass ?dry_grass - grass) 
      :precondition (and (inventory ?p ?grass) (not (dry ?grass)))
      :effect (and (inventory ?p ?dry_grass) (not (inventory ?p ?grass)))
   )

   (:action cluster_grass ; cluster gatherd dry grass to make tinder
      :parameters (?p - player ?dry_grass - grass ?grass_tinder - tinder) 
      :precondition (and (inventory ?p ?dry_grass))
      :effect (and (inventory ?p ?grass_tinder) (not (inventory ?p ?dry_grass)))
   )

   (:action break_branch ; break larger branches into smaller pieces
      :parameters (?p - player ?loc - location ?branch_piece - kindling) 
      :precondition (and (at ?p ?loc) (has_large_dry_branch ?loc))
      :effect (and (at ?branch_piece ?loc))
   )

   (:action log_oak ; log oak trees
      :parameters (?p - player ?loc - location ?axe - item ?oak_log - log) 
      :precondition (and (at ?p ?loc) (has_dry_oak ?loc) (inventory ?p ?axe))
      :effect (and (at ?oak_log ?loc) (dry ?oak_log))
   )

   (:action get_log ; log oak trees
      :parameters (?p - player ?loc - location ?oak_log - log) 
      :precondition (and (at ?p ?loc) (at ?oak_log ?loc) (dry ?oak_log))
      :effect (and (inventory ?p ?oak_log))
   )

   (:action stack_log ; stack dry logs to make enough fuel 
      :parameters (?p - player ?oak_log - log ?log_stack - fuel) 
      :precondition (and (inventory ?p ?oak_log))
      :effect (and (inventory ?p ?log_stack) (not (inventory ?p ?oak_log)))
   )

   (:action build_teepee ; build a teepee structure with tinder, kindling, and fuel logs
      :parameters (?p - player ?loc - location ?tinder - tinder ?kindling - kindling ?fuel_logs - fuel ?teepee - teepee)
      :precondition (and (inventory ?p ?tinder) (inventory ?p ?kindling) (inventory ?p ?fuel_logs) (has_dry_even_area ?loc) (at ?p ?loc))
      :effect (and (at ?teepee ?loc) (not (inventory ?p ?tinder)) (not (inventory ?p ?kindling)) (not (inventory ?p ?fuel_logs)))
   )

   (:action get_wood ; gather a flat piece of solid wood to start fire
      :parameters (?p - player ?loc - location ?wood - wood)
      :precondition (and (at ?p ?loc) (at ?wood ?loc))
      :effect (and (inventory ?p ?wood))
   )

   (:action get_branch ; gather a solid branch to start fire
      :parameters (?p - player ?loc - location ?branch - branch)
      :precondition (and (at ?p ?loc) (has_large_dry_branch ?loc))
      :effect (and (inventory ?p ?branch))
   )

   (:action ignite_wood ; ignite wood by plough it to create heat via friction
      :parameters (?p - player ?loc - location ?wood - wood ?branch - branch ?teepee - teepee)
      :precondition (and (at ?p ?loc) (at ?teepee ?loc) (inventory ?p ?wood) (inventory ?p ?branch))
      :effect (and (ignited ?wood))
   )

   (:action ignite_teepee ; ignite teepe with ignited wood to start fire
      :parameters (?p - player ?loc - location ?wood - wood ?teepee - teepee)
      :precondition (and (at ?p ?loc) (at ?teepee ?loc) (inventory ?p ?wood) (ignited ?wood))
      :effect (and (ignited ?teepee))
   )

   (:action get_pot ; retrieve a pot to hold water
      :parameters (?p - player ?loc - location ?pot - pot)
      :precondition (and (at ?p ?loc) (at ?pot ?loc))
      :effect (and (inventory ?p ?pot))
   )

   (:action get_water ; retrieve some natural but possibly contaminated water
      :parameters (?p - player ?loc - location ?water - water ?pot - pot)
      :precondition (and (at ?p ?loc) (has_pond ?loc) (inventory ?p ?pot))
      :effect (and (inventory ?p ?water))
   )

   (:action boil_water ; boil water using a started fire pit
      :parameters (?p - player ?loc - location ?teepee - teepee ?water - water)
      :precondition (and (at ?p ?loc) (at ?teepee ?loc) (ignited ?teepee) (inventory ?p ?water))
      :effect (and (boiled ?water))
   )
)
"""

# Write the domain to a PDDL file
domain_filename = "domain-survive_in_the_woods.pddl"
write_to_file(domain, path, domain_filename)

## Problem 1 - Find tinder material

<center>
<img src="https://www.wikihow.com/images/thumb/e/ea/Survive-in-the-Woods-Step-20.jpg/v4-460px-Survive-in-the-Woods-Step-20.jpg" class="img-responsive" alt="Find tinder. (Image Copyright wikiHow, Inc)"/>
</center>

In [ ]:
problem1 = """
(define (problem find_tinder)
   (:domain survive_in_the_woods)

   (:objects
      npc - player
      camp path grassland bush oaks wilderness - location
      in out north south east west up down - direction
      axe - item
      grass dry_grass - grass
      branch - branch
      oak_log - log
      grass_tinder - tinder
      branch_piece - kindling
      log_stack - fuel
   )

    (:init
        (connected camp west path)
        (connected path east camp)
        (connected path north grassland)
        (connected grassland south path)
        (connected grassland north bush)
        (connected bush south grassland)
        (connected path west wilderness)
        (connected wilderness east path)
        (connected path south oaks)
        (connected oaks north path)
        (at npc camp)
        (at axe camp)
        (at grass grassland)
        (has_large_dry_branch bush)
        (has_dry_oak oaks)
    )

   (:goal (and (inventory npc grass_tinder)))
)
"""

# Write the problem to a PDDL file
problem1_filename = "problem1-find_tinder.pddl"
write_to_file(problem1, path, problem1_filename)

In [ ]:
!python -B pddl-parser/PDDL.py $path/$domain_filename $path/$problem1_filename

----------------------------
['define',
 ['domain', 'survive_in_the_woods'],
 [':requirements', ':strips', ':typing'],
 [':types',
  'grass',
  'branch',
  'log',
  'tinder',
  'kindling',
  'fuel',
  'teepee',
  'wood',
  'water',
  'pot',
  '-',
  'item',
  'player',
  'direction',
  'location'],
 [':predicates',
  ['has_large_dry_branch', '?loc', '-', 'location'],
  ['has_dry_oak', '?loc', '-', 'location'],
  ['has_dry_even_area', '?loc', '-', 'location'],
  ['has_pond', '?loc', '-', 'location'],
  ['dry', '?item'],
  ['ignited', '?item'],
  ['boiled', '?item'],
  ['at', '?obj', '-', 'object', '?loc', '-', 'location'],
  ['inventory', '?player', '?item'],
  ['connected',
   '?loc1',
   '-',
   'location',
   '?dir',
   '-',
   'direction',
   '?loc2',
   '-',
   'location'],
  ['blocked',
   '?loc1',
   '-',
   'location',
   '?dir',
   '-',
   'direction',
   '?loc2',
   '-',
   'location']],
 [':action',
  'go',
  ':parameters',
  ['?dir',
   '-',
   'direction',
   '?p',
   '-',


In [ ]:
!python -B pddl-parser/planner.py $path/$domain_filename $path/$problem1_filename

Time: 0.008679866790771484s
plan:
go west npc camp path
go north npc path grassland
get grass npc grassland
cluster_grass npc grass grass_tinder



## Problem 2 - Gather kindling wood

<center>
<img src="https://www.wikihow.com/images/thumb/8/8f/Survive-in-the-Woods-Step-21.jpg/v4-460px-Survive-in-the-Woods-Step-21.jpg" class="img-responsive" alt="Gather kindling. (Image Copyright wikiHow, Inc)"/>
</center>

In [ ]:
problem2 = """
(define (problem gather_kindling)
   (:domain survive_in_the_woods)

   (:objects
      npc - player
      camp path grassland bush oaks wilderness - location
      in out north south east west up down - direction
      axe - item
      grass dry_grass - grass
      branch - branch
      oak_log - log
      grass_tinder - tinder
      branch_piece - kindling
      log_stack - fuel
   )

   (:init
      (connected camp west path)
      (connected path east camp)
      (connected path north grassland)
      (connected grassland south path)
      (connected grassland north bush)
      (connected bush south grassland)
      (connected path west wilderness)
      (connected wilderness east path)
      (connected path south oaks)
      (connected oaks north path)
      (at npc camp)
      (at axe camp)
      (at grass grassland)
      (has_large_dry_branch bush)
      (has_dry_oak oaks)
   )

   (:goal (and (inventory npc branch_piece)))
)
"""

# Write the problem to a PDDL file
problem2_filename = "problem2-gather_kindling.pddl"
write_to_file(problem2, path, problem2_filename)

In [ ]:
!python -B pddl-parser/PDDL.py $path/$domain_filename $path/$problem2_filename

----------------------------
['define',
 ['domain', 'survive_in_the_woods'],
 [':requirements', ':strips', ':typing'],
 [':types',
  'grass',
  'branch',
  'log',
  'tinder',
  'kindling',
  'fuel',
  'teepee',
  'wood',
  'water',
  'pot',
  '-',
  'item',
  'player',
  'direction',
  'location'],
 [':predicates',
  ['has_large_dry_branch', '?loc', '-', 'location'],
  ['has_dry_oak', '?loc', '-', 'location'],
  ['has_dry_even_area', '?loc', '-', 'location'],
  ['has_pond', '?loc', '-', 'location'],
  ['dry', '?item'],
  ['ignited', '?item'],
  ['boiled', '?item'],
  ['at', '?obj', '-', 'object', '?loc', '-', 'location'],
  ['inventory', '?player', '?item'],
  ['connected',
   '?loc1',
   '-',
   'location',
   '?dir',
   '-',
   'direction',
   '?loc2',
   '-',
   'location'],
  ['blocked',
   '?loc1',
   '-',
   'location',
   '?dir',
   '-',
   'direction',
   '?loc2',
   '-',
   'location']],
 [':action',
  'go',
  ':parameters',
  ['?dir',
   '-',
   'direction',
   '?p',
   '-',


In [ ]:
!python -B pddl-parser/planner.py $path/$domain_filename $path/$problem2_filename

Time: 0.010450124740600586s
plan:
go west npc camp path
go north npc path grassland
go north npc grassland bush
break_branch npc bush branch_piece
get branch_piece npc bush


## Problem 3 - Collect fuel

<center>
<img src="https://www.wikihow.com/images/thumb/7/74/Survive-in-the-Woods-Step-22.jpg/v4-460px-Survive-in-the-Woods-Step-22.jpg" class="img-responsive" alt="Collect fuel. (Image Copyright wikiHow, Inc)"/>
</center>

In [ ]:
problem3 = """
(define (problem collect_fuel)
   (:domain survive_in_the_woods)

   (:objects
      npc - player
      camp path grassland bush oaks wilderness - location
      in out north south east west up down - direction
      axe - item
      grass dry_grass - grass
      branch - branch
      oak_log - log
      grass_tinder - tinder
      branch_piece - kindling
      log_stack - fuel
   )

   (:init
      (connected camp west path)
      (connected path east camp)
      (connected path north grassland)
      (connected grassland south path)
      (connected grassland north bush)
      (connected bush south grassland)
      (connected path west wilderness)
      (connected wilderness east path)
      (connected path south oaks)
      (connected oaks north path)
      (at npc camp)
      (at axe camp)
      (at grass grassland)
      (has_large_dry_branch bush)
      (has_dry_oak oaks)
   )

   (:goal (and (inventory npc log_stack)))
)
"""

# Write the problem to a PDDL file
problem3_filename = "problem3-collect_fuel.pddl"
write_to_file(problem3, path, problem3_filename)

In [ ]:
!python -B pddl-parser/PDDL.py $path/$domain_filename $path/$problem3_filename

----------------------------
['define',
 ['domain', 'survive_in_the_woods'],
 [':requirements', ':strips', ':typing'],
 [':types',
  'grass',
  'branch',
  'log',
  'tinder',
  'kindling',
  'fuel',
  'teepee',
  'wood',
  'water',
  'pot',
  '-',
  'item',
  'player',
  'direction',
  'location'],
 [':predicates',
  ['has_large_dry_branch', '?loc', '-', 'location'],
  ['has_dry_oak', '?loc', '-', 'location'],
  ['has_dry_even_area', '?loc', '-', 'location'],
  ['has_pond', '?loc', '-', 'location'],
  ['dry', '?item'],
  ['ignited', '?item'],
  ['boiled', '?item'],
  ['at', '?obj', '-', 'object', '?loc', '-', 'location'],
  ['inventory', '?player', '?item'],
  ['connected',
   '?loc1',
   '-',
   'location',
   '?dir',
   '-',
   'direction',
   '?loc2',
   '-',
   'location'],
  ['blocked',
   '?loc1',
   '-',
   'location',
   '?dir',
   '-',
   'direction',
   '?loc2',
   '-',
   'location']],
 [':action',
  'go',
  ':parameters',
  ['?dir',
   '-',
   'direction',
   '?p',
   '-',


In [ ]:
!python -B pddl-parser/planner.py $path/$domain_filename $path/$problem3_filename

Time: 0.016072511672973633s
plan:
get axe npc camp
go west npc camp path
go south npc path oaks
log_oak npc oaks axe oak_log
get oak_log npc oaks
stack_log npc oak_log log_stack


## Problem 4 - Build a teepee structure 

<center>
<img src="https://www.wikihow.com/images/thumb/e/e7/Survive-in-the-Woods-Step-23.jpg/aid31352-v4-728px-Survive-in-the-Woods-Step-23.jpg" class="img-responsive" alt="Collect fuel. (Image Copyright wikiHow, Inc)" width="450"/>
</center>

In [ ]:
problem4 = """
(define (problem build_teepee)
   (:domain survive_in_the_woods)

   (:objects
      npc - player
      camp path grassland bush oaks wilderness - location
      in out north south east west up down - direction
      axe - item
      grass dry_grass - grass
      branch - branch
      oak_log - log
      grass_tinder - tinder
      branch_piece - kindling
      log_stack - fuel
      teepee_structure - teepee
   )

   (:init
      (connected camp west path)
      (connected path east camp)
      (connected path north grassland)
      (connected grassland south path)
      (connected grassland north bush)
      (connected bush south grassland)
      (connected path west wilderness)
      (connected wilderness east path)
      (connected path south oaks)
      (connected oaks north path)
      (at npc camp)
      (at axe camp)
      (at grass grassland)
      (has_large_dry_branch bush)
      (has_dry_oak oaks)
      (has_dry_even_area camp)
   )

   (:goal (and (at teepee_structure camp)))
)
"""

# Write the problem to a PDDL file
problem4_filename = "problem4-build_teepee.pddl"
write_to_file(problem4, path, problem4_filename)

In [ ]:
!python -B pddl-parser/PDDL.py $path/$domain_filename $path/$problem4_filename

----------------------------
['define',
 ['domain', 'survive_in_the_woods'],
 [':requirements', ':strips', ':typing'],
 [':types',
  'grass',
  'branch',
  'log',
  'tinder',
  'kindling',
  'fuel',
  'teepee',
  'wood',
  'water',
  'pot',
  '-',
  'item',
  'player',
  'direction',
  'location'],
 [':predicates',
  ['has_large_dry_branch', '?loc', '-', 'location'],
  ['has_dry_oak', '?loc', '-', 'location'],
  ['has_dry_even_area', '?loc', '-', 'location'],
  ['has_pond', '?loc', '-', 'location'],
  ['dry', '?item'],
  ['ignited', '?item'],
  ['boiled', '?item'],
  ['at', '?obj', '-', 'object', '?loc', '-', 'location'],
  ['inventory', '?player', '?item'],
  ['connected',
   '?loc1',
   '-',
   'location',
   '?dir',
   '-',
   'direction',
   '?loc2',
   '-',
   'location'],
  ['blocked',
   '?loc1',
   '-',
   'location',
   '?dir',
   '-',
   'direction',
   '?loc2',
   '-',
   'location']],
 [':action',
  'go',
  ':parameters',
  ['?dir',
   '-',
   'direction',
   '?p',
   '-',


In [ ]:
!python -B pddl-parser/planner.py $path/$domain_filename $path/$problem4_filename

Time: 0.7594428062438965s
plan:
go west npc camp path
go north npc path grassland
go north npc grassland bush
break_branch npc bush branch_piece
get branch_piece npc bush
go south npc bush grassland
get grass npc grassland
go south npc grassland path
go south npc path oaks
cluster_grass npc grass grass_tinder
log_oak npc oaks branch_piece oak_log
get oak_log npc oaks
go north npc oaks path
go east npc path camp
stack_log npc oak_log log_stack
build_teepee npc camp grass_tinder branch_piece log_stack teepee_structure


## Problem 5 - Create a fire plough to start the fire  

In [ ]:
problem5 = """
(define (problem start_fire)
   (:domain survive_in_the_woods)

   (:objects
      npc - player
      camp path grassland bush oaks wilderness - location
      in out north south east west up down - direction
      axe - item
      grass dry_grass - grass
      branch - branch
      oak_log - log
      grass_tinder - tinder
      branch_piece - kindling
      log_stack - fuel
      teepee_structure - teepee
      flat_wood - wood
   )

   (:init
      (connected camp west path)
      (connected path east camp)
      (connected path north grassland)
      (connected grassland south path)
      (connected grassland north bush)
      (connected bush south grassland)
      (connected path west wilderness)
      (connected wilderness east path)
      (connected path south oaks)
      (connected oaks north path)
      (at npc camp)
      (at axe camp)
      (at grass grassland)
      (at flat_wood oaks)
      (has_large_dry_branch bush)
      (has_dry_oak oaks)
      (has_dry_even_area camp)
   )

   (:goal (and (ignited teepee_structure)))
)
"""

# Write the problem to a PDDL file
problem5_filename = "problem5-start_fire.pddl"
write_to_file(problem5, path, problem5_filename)

In [ ]:
!python -B pddl-parser/PDDL.py $path/$domain_filename $path/$problem5_filename

----------------------------
['define',
 ['domain', 'survive_in_the_woods'],
 [':requirements', ':strips', ':typing'],
 [':types',
  'grass',
  'branch',
  'log',
  'tinder',
  'kindling',
  'fuel',
  'teepee',
  'wood',
  'water',
  'pot',
  '-',
  'item',
  'player',
  'direction',
  'location'],
 [':predicates',
  ['has_large_dry_branch', '?loc', '-', 'location'],
  ['has_dry_oak', '?loc', '-', 'location'],
  ['has_dry_even_area', '?loc', '-', 'location'],
  ['has_pond', '?loc', '-', 'location'],
  ['dry', '?item'],
  ['ignited', '?item'],
  ['boiled', '?item'],
  ['at', '?obj', '-', 'object', '?loc', '-', 'location'],
  ['inventory', '?player', '?item'],
  ['connected',
   '?loc1',
   '-',
   'location',
   '?dir',
   '-',
   'direction',
   '?loc2',
   '-',
   'location'],
  ['blocked',
   '?loc1',
   '-',
   'location',
   '?dir',
   '-',
   'direction',
   '?loc2',
   '-',
   'location']],
 [':action',
  'go',
  ':parameters',
  ['?dir',
   '-',
   'direction',
   '?p',
   '-',


In [ ]:
!python -B pddl-parser/planner.py $path/$domain_filename $path/$problem5_filename

Time: 4.3513877391815186s
plan:
go west npc camp path
go north npc path grassland
go north npc grassland bush
break_branch npc bush branch_piece
get branch_piece npc bush
get_branch npc bush branch
go south npc bush grassland
get grass npc grassland
go south npc grassland path
go south npc path oaks
get flat_wood npc oaks
cluster_grass npc grass grass_tinder
log_oak npc oaks flat_wood oak_log
get oak_log npc oaks
go north npc oaks path
go east npc path camp
stack_log npc oak_log log_stack
build_teepee npc camp grass_tinder branch_piece log_stack teepee_structure
ignite_wood npc camp flat_wood branch teepee_structure
ignite_teepee npc camp flat_wood teepee_structure


## Problem 6 - Use fire to purify your water

In [ ]:
problem6 = """
(define (problem boil_water)
   (:domain survive_in_the_woods)

   (:objects
      npc - player
      camp path grassland bush oaks wilderness - location
      in out north south east west up down - direction
      axe - item
      grass dry_grass - grass
      branch - branch
      oak_log - log
      grass_tinder - tinder
      branch_piece - kindling
      log_stack - fuel
      teepee_structure - teepee
      flat_wood - wood
      pot - pot
      water - water
   )

   (:init
      (connected camp west path)
      (connected path east camp)
      (connected path north grassland)
      (connected grassland south path)
      (connected grassland north bush)
      (connected bush south grassland)
      (connected path west wilderness)
      (connected wilderness east path)
      (connected path south oaks)
      (connected oaks north path)
      (at npc camp)
      (at pot camp)
      (at axe camp)
      (at grass grassland)
      (at flat_wood oaks)
      (has_large_dry_branch bush)
      (has_dry_oak oaks)
      (has_dry_even_area camp)
      (has_pond grassland)
   )

   (:goal (and (inventory npc water) (boiled water)))
)
"""

# Write the problem to a PDDL file
problem6_filename = "problem6-boil_water.pddl"
write_to_file(problem6, path, problem6_filename)

In [ ]:
!python -B pddl-parser/PDDL.py $path/$domain_filename $path/$problem6_filename

----------------------------
['define',
 ['domain', 'survive_in_the_woods'],
 [':requirements', ':strips', ':typing'],
 [':types',
  'grass',
  'branch',
  'log',
  'tinder',
  'kindling',
  'fuel',
  'teepee',
  'wood',
  'water',
  'pot',
  '-',
  'item',
  'player',
  'direction',
  'location'],
 [':predicates',
  ['has_large_dry_branch', '?loc', '-', 'location'],
  ['has_dry_oak', '?loc', '-', 'location'],
  ['has_dry_even_area', '?loc', '-', 'location'],
  ['has_pond', '?loc', '-', 'location'],
  ['dry', '?item'],
  ['ignited', '?item'],
  ['boiled', '?item'],
  ['at', '?obj', '-', 'object', '?loc', '-', 'location'],
  ['inventory', '?player', '?item'],
  ['connected',
   '?loc1',
   '-',
   'location',
   '?dir',
   '-',
   'direction',
   '?loc2',
   '-',
   'location'],
  ['blocked',
   '?loc1',
   '-',
   'location',
   '?dir',
   '-',
   'direction',
   '?loc2',
   '-',
   'location']],
 [':action',
  'go',
  ':parameters',
  ['?dir',
   '-',
   'direction',
   '?p',
   '-',


In [ ]:
!python -B pddl-parser/planner.py $path/$domain_filename $path/$problem6_filename

Time: 31.526995182037354s
plan:
get pot npc camp
go west npc camp path
go north npc path grassland
go north npc grassland bush
break_branch npc bush branch_piece
get branch_piece npc bush
get_branch npc bush branch
go south npc bush grassland
get grass npc grassland
cluster_grass npc grass grass_tinder
get_water npc grassland water pot
go south npc grassland path
go south npc path oaks
get flat_wood npc oaks
log_oak npc oaks pot oak_log
get oak_log npc oaks
go north npc oaks path
go east npc path camp
stack_log npc oak_log log_stack
build_teepee npc camp grass_tinder branch_piece log_stack teepee_structure
ignite_wood npc camp flat_wood branch teepee_structure
ignite_teepee npc camp flat_wood teepee_structure
boil_water npc camp teepee_structure water


# Step 3: Create Annotations for your Domain.

Finally, you will annotate data and save a JSON file that links the phrases in the wikiHow article that you selected with the different elements of your PDDL elements.  

If anyone is interested in doing a term project focused on automatically converting wikiHow to PDDL, then we'll share this JSON data with your classmates.

We provide a seperate Colab Notebook for you to perform this annotation.